# Random Number Generator Worksheet

This worksheet is largely based on the book by Gentle, Numerical Recipes (3rd Edition), and primary papers.

## Introduction

This tutorial will give an introduction to Random Number Generators (or RNGs, for short).

RNG algorithms are really pseudo-random.  This means they are deterministic.   Randomness (Jaynes) is a statement about knowledge, not about causes.   pRNGs have causes, but you might not know them, or you might use them in a way so that they appear to be random, or random ''enough."

What are the basic properties of RNs?
On any interval, there is an equal probability to obtain any value.
For convience, we will take this interval to be $(0,1)$.   Note, we will generally exclude $0$ from the interval when considering RNs on a computer, since many algorithms will fail if $0$ is ever encountered.

At the very least, we expect our computer RNs to have the properties of abstract ones.

Mean: $$\int_0^1 dx x = \frac{1}{2}$$

Variance:  $$\langle x^2 \rangle - \langle x \rangle^2 = \frac{1}{3}-\frac{1}{4} = \frac{1}{12}$$

In practice, computers represent only a finite range of numbers.
This has implications for RNGs.    Any algorithm that is deterministic and shuffles through the numbers is bound to repeat itself.   Further, if our algorithm uses one number as the seed for the next, this repetition will mean that the sequence of pRNs repeats itself.   This hardly looks random.   Much of the art in developing good RNGs is knowing how to characterize or calculate the length of these cycles.   If the cycle is long enough, much longer than any sequence of RNs we ever use in practice, then there is hope that this sequence will have the desired properties of random numbers.  Identifying an algorithm with a long cycle is a priority, and possibly led to the focus on modular arithmetic, to be discussed shortly.

## Middle square method

The need for such algorithms only came about with the development of practical computers and Monte Carlo (MC) techniques.   Historically, we are placing ourselves in the 1940-1950s.
Von Neumann (who else?) used a method called the [middle square](https://en.wikipedia.org/wiki/Middle-square_method).   It can give us some insight to the problem we are facing.

The middle square algorithm is short and strange:
1. input a number of length $n$
2. square the number and zero-pad if not of length $2n$
3. output the "middle" number of length $n$
4. use this number as the random variate and the input for step 1

For example:  $14^2 = 196 \to 0196 \to 19; 19^2 = 0361 \to 36; 36^2 = 1296 \to 29; 29^2 = 0841 \to 84$, etc.

Von Neumann used larger $n$ than this, but you get the point.

For $n$=2, try all seeds (inputs) and identify the longest sequence (before a repeat).
For his longest sequence, calculate the mean and variance.   Then, identify the worst seeds (0 period).


In [ ]:
import numpy as np

In [ ]:
print("mean = ", np.mean(r))

In [ ]:
seed_number = int(input("Please enter a four-digit number:\n[####] "))
number = seed_number
already_seen = set()
counter = 0
seq = []
while number not in already_seen:
    counter += 1
    already_seen.add(number)
    number = int(str(number * number).zfill(8)[2:6])  # zfill adds padding of zeroes
    print(f"#{counter}: {number}")
    seq.append(number)

print(
    f"We began with {seed_number} and"
    f" have repeated ourselves after {counter} steps"
    f" with {number}"
)
r = np.array(seq) / 9999.0
print(np.mean(r))
print(np.var(r))
###STOP_SOLUTION

In [ ]:
import numpy as np

In [ ]:
# Simple integer generator based on modular arithmetic
def next_one(item):
    return (3 * item) % 31

Lets investigate some other properties of this RNG.
The pairs $(x_i,x_{i-1})$ lie on a plane.   Plot their pattern.

A better choice of $a$ and $m$ can give our RNG better properties.
Write a function for a new RNG with $a=65539$ and $m=2^{31}$.
BEGIN_SOLUTION

Generate a sequence of length 20002, and normalize to the range (0,1].
Use seed =``` 2**8-1```
Make a sequence of points $(x_{i-1},x_{i+1})$ that satisfy the condition $0.50 < x_i < 0.51$.
Using ```matplotlib```, make a 2-D scatterplot.
BEGIN_SOLUTION

END_SOLUTION Note the unexpected linear relations.

Make a 3-D plot (x,y,z) of the triplets $(x_{i-1},x_{i},x_{i+1})$.
Restrict yourself to 1000 triplets.
Can you see any patterns?

The above example is not just pedagogical.  In fact, this RNG was/is called `RANDU`.
The authors of Numerical Recipes (3rd ed), share this anecdote:
> Even worse, many early generators happened to make particularly bad choices for m and a. One infamous such routine, RANDU, with a = 65539 and m = 2^31, was widespread on IBM mainframe computers for many years, and widely copied onto other systems. One of us recalls as a graduate student producing a “random” plot with only 11 planes and being told by his computer center’s programming consultant that he had misused the random number generator: “We guarantee that each number is random individually, but we don’t guarantee that more than one of them is random.” That set back our graduate education by at least a year!

## XORshift RNGs

Modern RNGs still combine two algorithms to remove undesired correlations.
However, they use an independent algorithm for the first sequence, so as not to unwittingly combine two correlations that arise from the same `type` of algorithm.

One such popular algorithm is called `XORshift`.  Its properties are understood by studying the multiplication of 3 special kinds of 32- or 64-dimensional binary matrices, but it can be programmed easily using bit shift and XOR operations.  The resulting algorithm does not look anything like matrix multiplication, but it really is.  This is because a bit shift can be represented on an n-bit vector by a matrix with only ones on a sub-diagonal.   Thus, to right-shift a bit sequence $\beta = (b_1,b_2,\cdots,b_n) \to \beta^{'} = (0,b_1,b_2,\cdots,b_{n-1})$, you would right multiply by an $n\times n$ matrix with only $1$s above the diagonal:
$$
\begin{pmatrix}
0 & 1 & 0 & \cdots &  0 \\
0 & 0 & 1 & \cdots &  0 \\
0 & 0 & 0 & 1      &  0 \\
\vdots & \vdots & \vdots & \ddots &  1 \\
0 & 0 & 0 & 0      &                 0  \\
\end{pmatrix}
$$

Similarly, a left-shift matrix has only $1$s on a subdiagonal `below` the diagonal.
Finally, since these are binary matrices, all operations use  integer arithmetic modulo 2.

The claim is that the series of operations $\beta T, \beta T^2 , \cdots \beta T^{2^n-1}$, every possible $\beta$ is produced.
This means that the cycle has length $2^n-1$.

For the current implementations, $T = (1_n+L^a)(1_n+R^b)(1_n+L^c)$, an $n\times n$ binary matrix with $(a,b,c)$ bit-shifts left-right-left.
Only certain triplets $(a,b,c)$ have the desired property.

A minimal test that $T$ has the desired properties is to square $T$ $n$ times and check if it is equal to $T$.

Perform this test for the tuples $(1,3,10)$, $(5,17,13)$, and $(2,5,14)$ using 32-bit precision.
Which of these are suitable triplets?

In [ ]:
def make_matrix(a, b, c):
    from scipy.sparse import diags

    E = sp.sparse.eye(32, dtype=np.int32)
    T1 = diags([1], [a], shape=(32, 32), dtype=np.int32) + E
    T2 = diags([1], [-b], shape=(32, 32), dtype=np.int32) + E
    T3 = diags([1], [c], shape=(32, 32), dtype=np.int32) + E
    #
    return T1 @ T2 @ T3

Use (a,b,c) = (5,17,13)

Choose a number < 10**32-1 and represent it an np.array of length 32, i.e. `3 = [0,0,....0,1,1]`.

Right multiplity (mod 2) by T and convert the result back to an integer.

Now, starting with the integer i, perform the operations:
i = i ^ i>>a i = i ^ i<<b i = i ^ i<<c

Show the results are equivalent.
Hint: make sure your integer doesn't become int64!

In [ ]:
numb = np.array(2**30 - 3, dtype=np.int32)

In [ ]:
print(c), bin(c)
print(func(c, 32))

In [ ]:
b = func(numb, 32)

In [ ]:
print(f)

In [ ]:
# Need help or must use c-functions
Ran:
    def __init__(self, seed):
        self.v = np.array(4101842887655102017, dtype=np.ulonglong)
        self.u = np.array(np.ulonglong(seed) ^ self.v, dtype=np.ulonglong)
        self.w = np.array(1, dtype=np.ulonglong)
        self.int64()
        self.v = self.u
        self.int64()
        self.w = self.v
        self.int64()

    def int64(self):
        self.u = self.u * 2862933555777941757 + np.ulonglong(7046029254386353087)
        self.v ^= self.v >> 17
        self.v ^= self.v << np.uint64(31)
        self.v ^= self.v >> np.uint64(8)
        self.w = np.uint32(4294957665) * (np.uint32(self.w) & 0xFFFFFFFF) + (
            np.uint32(self.w) >> 32
        )
        x = np.uint64(self.u) ^ np.uint64(np.uint64(self.u) << np.uint64(21))
        x ^= x >> np.uint64(35)
        x ^= x << np.uint64(4)
        state = np.uint64(x + self.v) ^ np.uint64(self.w)
        return state

    def doub(self):
        return 5.42101086242752217e-20 * self.int64()

    def int32(self):
        return np.uint32(self.int64())

## Tests of Random Number Generators

Diehard, NIST references.
Craps test?